In [1]:
import pandas as pd
import json
import datetime
import os
import numpy as np
from haversine import haversine
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/Margaret_zips_for_LayborDay_20190821'

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [3]:
df_stores_wanted=pd.read_excel("./2019 Labor Day SOTF Marketing - Stores 8.20.19.xlsx",sheet_name="Sheet1",skiprows=3,dtype=str)
list_stores_wanted=df_stores_wanted["Store#"].unique().tolist()

del df_stores_wanted['Unnamed: 0']
del df_stores_wanted['Unnamed: 1']
del df_stores_wanted['Unnamed: 2']

In [4]:
excel_file_new_Ta=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_remove_8_closing_stores_JL_2019-08-20.xlsx")
excel_file_new_Ta.sheet_names

['unique_zips_full_footprint', 'view_by_TA', 'view_by_store']

In [5]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190801-135940-752.txt",sep="|",dtype=str)
df_store_list.head(2)

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
0,1,BERWICK SHOPPING CENTER,1983-09-15,NaN,2837 WINCHESTER PIKE,NaN,COLUMBUS,OH,43232-9301,-000000000082.914789,000000000039.913636,00000000029217.0000,00000000006692.0000
1,29,ATHENS SHOPPING CENTER,1984-03-26,NaN,918 E STATE ST,NaN,ATHENS,OH,45701-2188,-000000000082.069765,000000000039.337172,00000000026545.0000,00000000005694.0000


In [6]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190801-135940-752.txt",sep="|",dtype=str)
df_output=df_store_list[['location_id','latitude_meas','longitude_meas']]
df_output['latitude_meas']=df_output['latitude_meas'].astype(float)
df_output['longitude_meas']=df_output['longitude_meas'].astype(float)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
df_zips_by_store=excel_file_new_Ta.parse("view_by_store",dtype=str)
df_iter=df_zips_by_store[df_zips_by_store['location_id'].isin(list_stores_wanted)]

df_mapping_store_zips=pd.DataFrame(columns=["location_id","zip_cd","trans_flag"])

for ind, row in df_iter.iterrows():
    store_number=row['location_id']
    P_zips=eval(row['all_trans_P_zips'])
    df_P=pd.DataFrame({"zip_cd":P_zips},index=["P"]*len(P_zips)).reset_index().rename(columns={"index":"trans_flag"})
    
    S_zips=eval(row['all_trans_S_zips'])
    df_S=pd.DataFrame({"zip_cd":S_zips},index=["S"]*len(S_zips)).reset_index().rename(columns={"index":"trans_flag"})
    
    dist_10_zips=eval(row['zips_in_10'])
    df_10_miles=pd.DataFrame({"zip_cd":dist_10_zips},index=["zips_in_10"]*len(dist_10_zips)).reset_index().rename(columns={"index":"trans_flag"})
    
    df=pd.concat([df_P,df_S,df_10_miles])
    df['location_id']=store_number
    df_mapping_store_zips=df_mapping_store_zips.append(df)
    
    
    

In [8]:
df_output=pd.merge(df_mapping_store_zips,df_output,on="location_id",how="left")
df_output.shape



(434, 5)

In [9]:
def get_dist_miles(zip_cd_str,lat,long):
    try:
        return haversine([lat,long],zip_centers[zip_cd_str],miles=True)
    except:
        return np.nan
    
df_output['dist_miles']=df_output.apply(lambda x: get_dist_miles(x['zip_cd'],x['latitude_meas'],x['longitude_meas']),axis=1)
df_output=df_output.rename(columns={"location_id":"Store#"})
df_output=df_output[((df_output['trans_flag']=="P") | (df_output['dist_miles']<=20))]

In [10]:
df_output=pd.merge(df_stores_wanted,df_output,on="Store#",how="left")

In [11]:
df_output_unique=df_output[['zip_cd','trans_flag']].sort_values(['zip_cd','trans_flag'])

df_output_unique=df_output_unique.drop_duplicates("zip_cd").sort_values(["trans_flag",'zip_cd'])

In [12]:
writer=pd.ExcelWriter("./BL_zips_for_2019LayborDay_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_unique.to_excel(writer,"unique_zip_cd",index=False)
df_output.to_excel(writer,"zip_by_store",index=False)
writer.save()

In [13]:
df_store_list[df_store_list['location_id'].isin(list_stores_wanted)]

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
54,391,D & L PLZ,1990-11-12,NaN,5165 BROADWAY,NaN,DEPEW,NY,14043-4012,-000000000078.683772,000000000042.901356,00000000018352.0000,00000000002654.0000
116,1018,NaN,1995-11-04,NaN,21 S MAIN ST,NaN,JAMESTOWN,NY,14701-6634,-000000000079.236407,000000000042.093175,00000000023403.0000,00000000004836.0000
259,831,NaN,1994-04-28,NaN,5987 S TRANSIT RD,NaN,LOCKPORT,NY,14094-6305,-000000000078.694654,000000000043.137017,00000000022103.0000,00000000003406.0000
304,1178,COLUMBIA NORTHEAST SHOPPING CENTER,1997-10-12,NaN,9221 TWO NOTCH RD STE 30,NaN,COLUMBIA,SC,29223-6428,-000000000080.923807,000000000034.096512,00000000016031.0000,00000000002109.0000
426,1692,WESMARK PLAZA,2002-07-11,NaN,1121 BROAD ST,NaN,SUMTER,SC,29150-1902,-000000000080.374460,000000000033.954456,00000000021874.0000,00000000003060.0000
511,1404,D&F PLZ,1998-11-22,NaN,1170 CENTRAL AVE,NaN,DUNKIRK,NY,14048-3424,-000000000079.328080,000000000042.458276,00000000023574.0000,00000000004867.0000
536,1520,NaN,2000-07-13,NaN,1980 RIDGE RD,NaN,WEST SENECA,NY,14224-3312,-000000000078.778207,000000000042.830778,00000000022480.0000,00000000004514.0000
561,1629,YOUNGMANN PLAZA,2001-10-12,NaN,750 YOUNG ST,NaN,TONAWANDA,NY,14150-4101,-000000000078.858223,000000000043.001400,00000000026512.0000,00000000005922.0000
603,1762,SOUTH OGDEN SHOPPING CENTER,2003-07-18,NaN,698 S OGDEN ST,NaN,BUFFALO,NY,14206-2317,-000000000078.802542,000000000042.881185,00000000023361.0000,00000000003781.0000
635,1851,DELAWARE PARK CENTRE,2004-08-13,NaN,502 N UNION ST,NaN,OLEAN,NY,14760-2619,-000000000078.428947,000000000042.085270,00000000020350.0000,00000000003675.0000
